In [ ]:
import sys, numpy, pandas, polars, matplotlib, seaborn, sklearn
print(f"✅ Python {sys.version.split()[0]} | NumPy {numpy.__version__} | Pandas {pandas.__version__} | Polars {polars.__version__} | Matplotlib {matplotlib.__version__} | Seaborn {seaborn.__version__} | Scikit-learn {sklearn.__version__}")

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

ふざけたエラーに遭遇。
解決策...[自分用メモ：OpenAIのライブラリ使うと「TypeError: Client.__init__() got an unexpected keyword argument 'proxies'」が出る - Qiita](https://qiita.com/hatman621221/items/fafaf4ac0e906616e8bd)

In [1]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは私はジョンといいます。"},
    ],
)

print(response.to_json(indent=2))

{
  "id": "chatcmpl-ByN3qPlS6TnWflRVxXv2Eoy5w2vxy",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "こんにちは、ジョンさん！お会いできて嬉しいです。どんなことについてお話ししましょうか？",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      }
    }
  ],
  "created": 1753728026,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_197a02a720",
  "usage": {
    "completion_tokens": 26,
    "prompt_tokens": 25,
    "total_tokens": 51,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}


# Function Calling

## 概要

利用可能な関数を伝えておいて、LLMに関数を使いたいという判断をさせる機能。

In [4]:
import json

def get_current_weather(location, unit="fahrenheit"):
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters":{
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [5]:
from openai import OpenAI

client = OpenAI()

messages = [
    {"role": "user", "content": "東京の天気はどうですか？"},
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

print(response.to_json(indent=2))

{
  "id": "chatcmpl-ByNql8AqNI3x3KSQECqXHMJsMQOU3",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_ijDcqnaT1gTexDUHPS2NdNmi",
            "function": {
              "arguments": "{\"location\":\"Tokyo, Japan\",\"unit\":\"celsius\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ],
        "annotations": []
      }
    }
  ],
  "created": 1753731059,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_a288987b44",
  "usage": {
    "completion_tokens": 22,
    "prompt_tokens": 81,
    "total_tokens": 103,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens

今までは以下のように、contentに回答が入っていた。

```json
{
  "id": "chatcmpl-ByN3qPlS6TnWflRVxXv2Eoy5w2vxy",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "こんにちは、ジョンさん！お会いできて嬉しいです。どんなことについてお話ししましょうか？",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      }
    }
  ],
  ...
}
```

しかし今回はcontentが空で、代わりにtool_calls要素が追加され、そこにやりたいことが書いてある。

```json
{
  "id": "chatcmpl-ByNql8AqNI3x3KSQECqXHMJsMQOU3",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_ijDcqnaT1gTexDUHPS2NdNmi",
            "function": {
              "arguments": "{\"location\":\"Tokyo, Japan\",\"unit\":\"celsius\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ],
        "annotations": []
      }
    }
  ],
...
      "rejected_prediction_tokens": 0
    }
  }
}
```